In [5]:
import os
import sys
import json
import nest_asyncio, asyncio
import logging
from pathlib import Path
import pyarrow as pa

import re, ast

from datasets import Dataset, Features, Sequence, Value, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer
from google import genai
from google.genai import types

GEMINI_API_KEY  = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise RuntimeError("GEMINI_API_KEY env var not set – please `export GEMINI_API_KEY=…` before launching.")
    
MODEL = "gemini-2.0-flash"
model_name = "gemini-2.0"

tokenizer = AutoTokenizer.from_pretrained(
    "/scratch/gpfs/JHA/mb5157/tokenizers/biomedbert_fast_tokenizer"
)


SAVE_CHUNK      = 1000               # save result each save every BATCH_SIZE examples
PRINTOUT        = True
TAKE_SUBSET     = False
SUBSET_SIZE      = 50
BATCH_SIZE      = 128               # how many concurrent Gemini calls
TIME_SLEEP = 0.05

TTL = '259200s' # for cache
PRINT_RAW_RESPONSE = False


# set MEANING_EXPL_general

In [6]:
PREFIX = Path('/scratch/gpfs/JHA/mb5157/repos/graphmert')
PREDICTION_SUFFIX = 'predictions/top_20/'

In [7]:
logging.basicConfig(level=logging.INFO, force=True)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
fmt = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
handler.setFormatter(fmt)
logger.addHandler(handler)

# Quiet down other noisy loggers
logging.getLogger("google_genai.models").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [ ]:
from importlib import reload

from combine_tails_prompts import (
    SYSTEM_CONTEXT, 
    example_user_1, example_assistant_1, example_explanation_1,
    example_user_2, example_assistant_2, example_explanation_2,
    example_user_3, example_assistant_3, example_explanation_3,
    example_user_4, example_assistant_4, example_explanation_4,
    example_user_5, example_assistant_5, example_explanation_5,
    example_user_6, example_assistant_6, example_explanation_6,
    example_user_7, example_assistant_7, example_explanation_7,
    example_user_8, example_assistant_8, example_explanation_8,
    example_user_9, example_assistant_9, example_explanation_9,
)

few_shot_examples = [
    {"user_query": example_user_1, "model_answer": example_assistant_1, "model_explanation": example_explanation_1},
    {"user_query": example_user_2, "model_answer": example_assistant_2, "model_explanation": example_explanation_2},
    {"user_query": example_user_3, "model_answer": example_assistant_3, "model_explanation": example_explanation_3},
    {"user_query": example_user_4, "model_answer": example_assistant_4,  "model_explanation": example_explanation_4},
    {"user_query": example_user_5, "model_answer": example_assistant_5,  "model_explanation": example_explanation_5},
    {"user_query": example_user_6, "model_answer": example_assistant_6,  "model_explanation": example_explanation_6},
    {"user_query": example_user_7, "model_answer": example_assistant_7,  "model_explanation": example_explanation_7},
    {"user_query": example_user_8, "model_answer": example_assistant_8,  "model_explanation": example_explanation_8},
    {"user_query": example_user_9, "model_answer": example_assistant_9,  "model_explanation": example_explanation_9},
]

STRUCTURED_CONTENT_FEW_SHOTS = [
    {"role": "user", "parts": [{"text": "I will provide you with examples"}]},
    {"role": "model", "parts": [{"text": "Understood — send the sample and I’ll output relations"}]},
]

for ex in few_shot_examples:
    STRUCTURED_CONTENT_FEW_SHOTS.append({
        "role": "user",
        "parts": [{"text": ex["user_query"]}]
    })
    # example model answer
    STRUCTURED_CONTENT_FEW_SHOTS.append({
        "role": "model",
        "parts": [{"text": ex["model_answer"]}]
    })

    if "model_explanation" in ex:
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "user",
            "parts": [{"text": "Explanation of the previous output:"}]
        })
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "model",
            "parts": [{"text": ex["model_explanation"]}]
        })

    if "model_explanation_negative" in ex:
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "user",
            "parts": [{"text": "Explanation of what is wrong with the previous output:"}]
        })
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "model",
            "parts": [{"text": ex["model_explanation_negative"]}]
        })

STRUCTURED_CONTENT_FEW_SHOTS.append({
    "role": "user",
    "parts": [{"text": "**End of examples**.\nNow read the actual input:"}]
})

In [6]:
def extract_rightmost_list(response: str) -> list:
    # 1) Strip any lines that start with ``` (with or without language tag)
    #    but kepp what's between them.
    response = re.sub(r'(?m)^```.*\n?', "", response)

    # 2) Find all [ ... ] spans (non-greedy, across newlines)
    matches = re.findall(r"\[.*?\]", response, flags=re.DOTALL)
    if not matches:
        return []
    candidate = matches[-1]

    # 3) Try JSON, then Python literal
    try:
        return json.loads(candidate)
    except json.JSONDecodeError:
        pass
    try:
        return ast.literal_eval(candidate)
    except Exception:
        return []

In [7]:
async def gemini_call(client, user_query: str, count_only, cache=None, cache_system_prompt=True, max_retries: int=3) -> str:
    """
    Wrap Gemini call in a retrying try/except so it never blows up.
    count_only: mock call for counting input tokens.
    """
    global input_tokens, output_tokens
    # count_tokens is mock request, generate_content is actual paid request
    call_fn = client.aio.models.count_tokens if count_only else client.aio.models.generate_content

    config = types.GenerateContentConfig(cached_content=cache.name) if cache is not None else None
    if cache_system_prompt:
        contents = [{
                "role": "user",
                "parts": [{"text": user_query}]
            }]
    else:
        contents = []
        contents.extend(STRUCTURED_CONTENT_FEW_SHOTS)
        contents.append({
                "role": "user",
                "parts": [{"text": user_query}]
            })
        
    for attempt in range(1, max_retries + 1):
        try:            
            resp = await call_fn(model=MODEL, contents=contents, config=config)
            if count_only:
                # count_tokens returns an int of prompt tokens
                input_tokens += resp.total_tokens
                return ""

            # real response path
            metadata = resp.usage_metadata
            pt = metadata.prompt_token_count or 0
            cc = metadata.cached_content_token_count or 0
            ct = metadata.candidates_token_count or 0
            input_tokens += pt - cc
            output_tokens += ct
            return getattr(resp, "text", "") or ""

        except Exception as e:
            # catch HTTPError, ValidationError, ServerError, etc.
            logger.warning(f"Gemini call attempt {attempt}/{max_retries} failed: {e!r}")
            if attempt < max_retries:
                await asyncio.sleep(0.5 * attempt)    # simple backoff
            else:
                logger.error(f"All {max_retries} Gemini attempts failed for query: {user_query[:50]!r}")
                return ""  # give up


async def process_batch(client, cache, slice_of_examples, count_only, cache_system_prompt, print_raw_response=PRINT_RAW_RESPONSE):
    """
    Fire off up to BATCH_SIZE concurrent requests.
    """
    tasks = []
    for ex in slice_of_examples:
        seq  = tokenizer.decode(ex["input_ids"], skip_special_tokens=True)
        head = ex['head']
        relation = ex['relation']
        predictions = ex['predictions']
        uq = f"{seq}\nhead: {head}\nrelation: {relation}\npredictions: {predictions}" 
        tasks.append(gemini_call(client, uq, count_only, cache, cache_system_prompt))
        await asyncio.sleep(TIME_SLEEP)
        
    raw_resps = await asyncio.gather(*tasks)

    if print_raw_response:
        for raw_resp in raw_resps:
            print(raw_resp)
    
    cleaned = [extract_rightmost_list(r) for r in raw_resps]
    return cleaned


async def create_cache(client, count_only):
    global input_tokens, cached_tokens

    if count_only: # free, only for input token count
        cache_contents_mock = [{"role": "user", "parts": [{"text": SYSTEM_CONTEXT}]}]
        cache_contents_mock.extend(STRUCTURED_CONTENT_FEW_SHOTS)
        
        cache_call_tokens = await client.aio.models.count_tokens(
            model=MODEL,
            contents=cache_contents_mock,
        )
        cached_tokens += cache_call_tokens.total_tokens
        return None
        
    else: # paid call
        cache_config = types.CreateCachedContentConfig(
                system_instruction=SYSTEM_CONTEXT,
                contents=STRUCTURED_CONTENT_FEW_SHOTS,
                ttl=TTL
            )
        cache = await client.aio.caches.create(
            model=MODEL,
            config=cache_config,  
        )
        logger.info("created cache")
        # count actual tokens cached in a paid request 
        cached_tokens_call = cache.usage_metadata.total_token_count
        cached_tokens += cached_tokens_call
        return cache
        

async def run_all(client, dataset: Dataset, chunk_start: int, chunk_end: int, cache=None, count_only=True):
    """
    Split into batches
    """
    n = len(dataset)
    all_resp = [None] * n

    for start in range(0, n, BATCH_SIZE):
        end = min(start + BATCH_SIZE, n)
        slice_ds = dataset.select(range(start, end))
        cleaned  = await process_batch(client, cache, slice_ds, count_only, cache_system_prompt)
        all_resp[start:end] = cleaned
        if PRINTOUT:
            logger.info(f"Gemini: processed {end}/{n}")
            
    return all_resp

In [8]:
async def main(model_path: str, count_only=True, cache_system_prompt=True, start_idx: int = 0, end_idx=None):
    dataset_dir = PREFIX / model_path / PREDICTION_SUFFIX
    ds_full = Dataset.load_from_disk(str(dataset_dir))
    if TAKE_SUBSET:
        overall_end = min(start_idx + SUBSET_SIZE, len(ds_full))
    else:
        overall_end = len(ds_full) if end_idx is None else min(len(ds_full), end_idx)

    client = genai.Client(api_key=GEMINI_API_KEY)
    cache = None

    try:
        if cache_system_prompt:
            cache = await create_cache(client, count_only)
            
        for chunk_start in range(start_idx, overall_end, SAVE_CHUNK):
            chunk_end = min(chunk_start + SAVE_CHUNK, overall_end)
            ds_chunk = ds_full.select(range(chunk_start, chunk_end))
            tails = await run_all(client, ds_chunk, chunk_start, chunk_end, cache, count_only=count_only)
            
            assert len(tails) == ds_chunk.num_rows, f"{len(tails)=} vs {ds_chunk.num_rows=}"
            if not count_only:
                # coerce any stray string into a single‐element list
                fixed = [[str(x) for x in t] if isinstance(t, list) else [str(t)] for t in tails]
                tails_arr = pa.array(fixed, type=pa.list_(pa.string()))
                ds_chunk = ds_chunk.add_column("tails", fixed)
                if TAKE_SUBSET:
                    save_name = f"{model_name}_subset_{SUBSET_SIZE}_{chunk_start}-{chunk_end}"
                else:
                    save_name = f"{model_name}_{chunk_start}-{chunk_end}"
        
                out_dir = os.path.join(str(dataset_dir), save_name)
                ds_chunk.save_to_disk(out_dir)
                logger.info(f"Saved chunk {chunk_start}-{chunk_end} to {out_dir}")
    finally:
        if cache is not None:
            await client.aio.caches.delete(name=cache.name)

In [9]:
input_tokens = 0
output_tokens = 0
cached_tokens = 0

count_only = False # True means no real requests
cache_system_prompt = True

model_path = Path('outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/')
await main(model_path, count_only=count_only, cache_system_prompt=cache_system_prompt, start_idx=0, end_idx=None)

print('===finished===')
print('input_tokens:', input_tokens)
print('output_tokens:', output_tokens)
print('cached_tokens', cached_tokens)

2025-08-12 14:59:18,715 INFO created cache


INFO:__main__:created cache


2025-08-12 14:59:27,304 INFO Gemini: processed 128/1000


INFO:__main__:Gemini: processed 128/1000


2025-08-12 14:59:35,771 INFO Gemini: processed 256/1000


...

2025-08-13 00:59:12,594 INFO Gemini: processed 906/906


INFO:__main__:Gemini: processed 906/906
Saving the dataset (1/1 shards): 100%|██████████| 906/906 [00:00<00:00, 163175.86 examples/s]

2025-08-13 00:59:12,611 INFO Saved chunk 2315000-2315906 to /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_2315000-2315906



INFO:__main__:Saved chunk 2315000-2315906 to /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_2315000-2315906


===finished===
input_tokens: 98268799
output_tokens: 5161432
cached_tokens 4099


# Unite datasets batches together

In [19]:
llm_name = 'gemini-2.0'

dataset_path = PREFIX / model_path / PREDICTION_SUFFIX
output_path = os.path.join(dataset_path, f'{llm_name}_all')

batch_size = 1000
original_dataset_size = 2315906
start_idx = 0
end_idx = 0


datasets = []
while end_idx < original_dataset_size:
    end_idx = min(start_idx + batch_size, original_dataset_size)
    path = os.path.join(dataset_path, f"{llm_name}_{start_idx}-{end_idx}")
    dataset = load_from_disk(path)
    print(f'loaded from {path}')
    dataset = dataset.select_columns(['id', 'input_ids', 'head', 'relation', 'predictions', 'tails'])
    datasets.append(dataset)
    start_idx += batch_size
    

united_dataset = concatenate_datasets(datasets)
united_dataset.save_to_disk(output_path)

loaded from /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_0-1000
loaded from /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_1000-2000
loaded from /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_2000-3000
loaded from /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_3000-4000
loaded from /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_4000-5000
loaded from /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_5000-6000
loaded from /scratc

Saving the dataset (4/4 shards): 100%|██████████| 2315906/2315906 [00:02<00:00, 803147.12 examples/s]


In [40]:
# remove hallicinated tails -- not from the prediction

import csv

def filter_tails(example):
    preds = set(example['predictions'].split())
    kept, removed, removed_missing = [], [], []
    for tail in example['tails']:
        tl = tail.lower()
        toks = tokenizer.tokenize(tl)
        if all(tok in preds for tok in toks):
            kept.append(tl)
        else:
            removed.append(tl)
            removed_missing.append([tok for tok in toks if tok not in preds])
    return {
        'tails': kept,
        'removed_tails': removed, # the actual tail strings that were filtered out
        'removed_missing_tokens': removed_missing, # tokens that triggered the removal
    }

clean_dataset = united_dataset.map(
    filter_tails, num_proc=20, desc="filter tails"
)

# ---- totals ----
kept_total    = sum(len(x) for x in clean_dataset['tails'])
removed_total = sum(len(x) for x in clean_dataset['removed_tails'])
total_before  = kept_total + removed_total

print(f"tails kept: {kept_total}")
print(f"tails removed: {removed_total}")
print(f"tails total (before filtering): {total_before}")

clean_dataset.select_columns(['id', 'input_ids', 'head', 'relation', 'predictions', 'tails']).save_to_disk(f'{output_path}_clean')
print(f'saved to {output_path}_clean')

tails kept: 3998671
tails removed: 526803
tails total (before filtering): 4525474


Saving the dataset (4/4 shards): 100%|██████████| 2315906/2315906 [00:02<00:00, 977254.03 examples/s]

saved to /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_all_clean


In [41]:
log_path = os.path.join(output_path, f'removed_tails.csv')
with open(log_path, 'w', newline='', encoding='utf-8') as f:
    w = csv.writer(f)
    header = ['row_index', 'id', 'removed_tail', 'missing_tokens']
    w.writerow(header)
    has_id = 'id' in clean_dataset.column_names
    for i, row in enumerate(clean_dataset):
        rid = row['id'] if has_id else ''
        for tail, missing in zip(row['removed_tails'], row['removed_missing_tokens']):
            w.writerow([i, rid, tail, ' '.join(missing)])

print(f"saved to {output_path_clean}\nlog: {log_path}")

saved to /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_clean
log: /scratch/gpfs/mb5157/repos/graphmert/graphmert/outputs/350abstracts/gemini/score0.57/span/rel36/bs256_lr_0.0004/predictions/top_20/gemini-2.0_all/removed_tails.csv
